In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline

df1 = pd.read_csv('Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv', low_memory=False)
df2 = pd.read_csv('Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv', low_memory=False)

df = pd.concat([df1,df2])

del df['Timestamp'] 

df['keep'] = df.iloc[:, 0:-1].apply(
    lambda row: pd.to_numeric(row, errors='coerce').notnull().all(),
    axis=1
)

df = df.replace([np.inf, -np.inf], np.nan)
df = df[df['keep']]
del df['keep']

df = df.dropna()
df = df.reset_index(drop=True)

X = df.iloc[:, :-1].values #features are all but last column
y = df.iloc[:, 78].values #label is last column

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[241762     15      8   2835]
 [     2   8300      0      0]
 [     2      0   2187      0]
 [ 12592      0      1   6042]]
                       precision    recall  f1-score   support

               Benign       0.95      0.99      0.97    244620
DoS attacks-GoldenEye       1.00      1.00      1.00      8302
DoS attacks-Slowloris       1.00      1.00      1.00      2189
        Infilteration       0.68      0.32      0.44     18635

             accuracy                           0.94    273746
            macro avg       0.91      0.83      0.85    273746
         weighted avg       0.93      0.94      0.93    273746

